# Interactive spec creation

This notebook is intended to make it easier to generate ground truth for trips to put into the evaluation spec files. Putting in a bunch of trips into a spec file and trying to fill in all of them leads to wasted time, specially if we have a lot of relations, since we must query OSM over and over to pull the data to populate the coordinates. And then we have to visualize all the trips at once as part of the validation, so if we find any errors, we need to regenerate the coordinates for all trips.

This notebook allows us to interactively develop individual specs before copy-pasting them into the overall spec file.

### These initial cells are for setup and should not need to be changed

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

#### Let's see what we can do with the OSM API directly

In [ ]:
import autofill_eval_spec as aes

In [ ]:
import folium
import folium.features as fof
import folium.utilities as ful
import branca.element as bre

In [ ]:
import polyline as pl
import json

In [ ]:
import osmapi

In [ ]:
osm = osmapi.OsmApi()

In [ ]:
def lonlat_swap(lon_lat):
    return list(reversed(lon_lat))

In [ ]:
def get_one_marker(loc, disp_color):
    if loc["geometry"]["type"] == "Point":
        curr_latlng = lonlat_swap(loc["geometry"]["coordinates"])
        return folium.Marker(curr_latlng, icon=folium.Icon(color=disp_color),
                  popup="%s" % loc["properties"]["name"])
    elif loc["geometry"]["type"] == "Polygon":
        assert len(loc["geometry"]["coordinates"]) == 1,\
            "Only simple polygons supported!"
        curr_latlng = [lonlat_swap(c) for c in loc["geometry"]["coordinates"][0]]
        # print("Returning polygon for %s" % curr_latlng)
        return folium.PolyLine(curr_latlng, color=disp_color, fill=disp_color,
                  popup="%s" % loc["properties"]["name"])

In [ ]:
def get_marker(loc, disp_color):
    if type(loc) == list:
        return [get_one_marker(l, disp_color) for l in loc]
    else:
        print("Found single entry, is this expected?")
        return [get_one_marker(loc, disp_color)]

In [ ]:
def add_waypoint_markers(waypoint_coords, curr_map):
    for i, wpc in enumerate(waypoint_coords["geometry"]["coordinates"]):
        folium.map.Marker(
            lonlat_swap(wpc), popup="%d" % i,
            icon=fof.DivIcon(class_name='leaflet-div-icon')).add_to(curr_map)

def get_map_for_travel_leg(trip):
    curr_map = folium.Map()
    # print([loc for loc in trip["start_loc"]])
    print([loc for loc in trip["end_loc"]])
    [get_one_marker(loc, "green").add_to(curr_map) for loc in trip["start_loc"]]
    [get_one_marker(loc, "red").add_to(curr_map) for loc in trip["end_loc"]]
    
    # iterate over all reroutes
    for rc in trip["route_coords"]:
        coords = rc["geometry"]["coordinates"]
        print("Found %d coordinates for the route" % (len(coords)))
        
        latlng_coords = [lonlat_swap(c) for c in coords]
        folium.PolyLine(latlng_coords, popup="%s: %s" % (trip["mode"], trip["name"])).add_to(curr_map)
        
        for i, c in enumerate(latlng_coords):
            folium.CircleMarker(c, radius=5, popup="%d: %s" % (i, c)).add_to(curr_map)
            
        curr_map.fit_bounds(ful.get_bounds(latlng_coords))
    
    return curr_map

In [ ]:
def get_map_for_shim_leg(trip):
    curr_map = folium.Map()
    for loc in trip["loc"]:
        mkr = get_one_marker(loc, "purple")
        mkr.add_to(curr_map)
        curr_map.fit_bounds(mkr.get_bounds())
    return curr_map

# This is the important part

Instructions:
- Experiment with methods to generate start and end polygons or polylines (either here or using http://geojson.io or https://open-polyline-decoder.60devs.com/)
- Edit the cell below and use them to create a spec leg
- Run the cells below that to fill it in and display it
- Tweak the various fill/population methods until the trip looks right
- The leg is saved to a file (`/tmp/curr_leg.json`) every time
- Once the leg is final, include the text *in the file* into your spec
- DO NOT copy and paste directly from the notebook; the JSON can be finicky about the formatting
- Rinse and repeat

### Option 1: Using an OSM relation and a start and end node

Used to create the bus route in the sample spec

In [ ]:
# Using a public transit relation and start and end stops from OSM
# This is the 52 bus line from El Camino/Castro to Foothill College
route_coords = aes.get_coords_for_relation(rid=9937346, start_node=6703651466, end_node=343648647)
pl.encode(route_coords)

## Option 2a: Using OSRM with OSM nodes

- Used to create the walking route in the sample spec (to bus stop and from bus stop)
- The Foothill version was post-edited with geojson.io since OSRM persisted in doubling back

In [ ]:
t = {
    "mode": "WALKING", # or CAR, BICYCLING or BUS
    "route_waypoints": [6426801140, 2338378009, 2338378003]
}
route_coords = aes.get_route_from_osrm(t, start_coords=aes.node_to_geojson_coords(5301175134), end_coords=aes.node_to_geojson_coords(2161940025))
pl.encode(route_coords)

### Example of re-encoding the polyline after editing

- Copy paste the old coords into geojson.io
- edit
- paste the new coords back here
- encode into the new polyline

In [ ]:
# [lonlat_swap(c) for c in route_coords]

In [ ]:
# new_coords = []
# pl.encode([lonlat_swap(c) for c in new_coords])

## Option 2b: Using OSRM with coordinate waypoints

Used to create the car trip in the sample spec
- use geojson.io to find start and end points in the polygon 
- use geojson.io to create the waypoint linestring

Post-edited in geojson.io to avoid weird routing with OSRM

In [ ]:
t = {
    "mode": "CAR", # or CAR, BICYCLING or BUS
    "waypoint_coords":     
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [-122.09133803844452, 37.37410486690446],
          [-122.08707869052887, 37.374087814725534],
          [-122.08697140216826, 37.3750235473139],
          [-122.08697944879532, 37.3772040360451],
          [-122.08701968193054, 37.38092329624445],
          [-122.08611845970152, 37.38176942882687]
        ]
      }
    }
}
start_lonlat = [-122.13025152683258, 37.35973490013966]
end_lonlat = [-122.08356499671936, 37.39037087962563]
route_coords = aes.get_route_from_osrm(t, start_coords=start_lonlat, end_coords=end_lonlat)
pl.encode(route_coords)

### Tweak if needed

In [ ]:
# [lonlat_swap(c) for c in route_coords]

In [ ]:
# new_coords = []
# pl.encode([lonlat_swap(c) for c in new_coords])

In [ ]:
import importlib

In [ ]:
importlib.reload(aes)

## Spec we are working on

In [ ]:
curr_leg_spec = \
{
                "id": "campus_walk",
                "name": "Walk on the Foothill campus",
                "mode": "WALKING",
                "start_loc": {
                    "type": "Feature",
                    "properties": {
                        "name": "Foothill pool"
                    },
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [
                            [
            [
              -122.12678946554661,
              37.3624887707069
            ],
            [
              -122.1268632262945,
              37.36245146328467
            ],
            [
              -122.12655209004879,
              37.362050140841276
            ],
            [
              -122.12643943727016,
              37.36215779992784
            ],
            [
              -122.12671637535095,
              37.36252447922224
            ],
            [
              -122.12678946554661,
              37.3624887707069
            ]
                            ]
                        ]
                    }
                },
                "end_loc": {
                    "type": "Feature",
                    "properties": {
                        "name": "Foothill fountain"
                    },
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [
                            [
                                [ -122.12924435734747, 37.36135941352246 ],
                                [ -122.12901771068573, 37.36108013616577 ],
                                [ -122.12868243455887, 37.361213379309085 ],
                                [ -122.12891444563866, 37.361499051811045 ],
                                [ -122.12924435734747, 37.36135941352246 ]
                            ]
                        ]
                    }
                },
                "polyline": "mj`cFd{khVIHEGJIKHk@uAGOVQBFDHBHDH@BBFHRt@i@BFDHRRDDHFFJDJn@~AN`@L^_@Va@TOb@D\\Pb@JRF^Df@????f@xA????j@]ULUNg@yAEg@G_@KSQc@E]Nc@`@U^W"
            }

### Write it out before filling it in

In [ ]:
json.dump(curr_leg_spec, open("/tmp/curr_leg.json", "w"), indent=4)

### Fill it in

In [ ]:
curr_leg_spec = aes.validate_and_fill_leg(curr_leg_spec, "2020-01-01", "2021-10-10")

In [ ]:
# Dump filled value to copy-paste into geojson.io as needed
# json.dumps(curr_leg_spec["route_coords"][0])

### Validate it

In [ ]:
get_map_for_travel_leg(curr_leg_spec)

### Insert into spec

- If everything looks good, insert the leg from `/tmp/curr_leg.json` into the spec
- If something looks wrong, keep tweaking

Sat down in San Jose: 12:28pm
Got up from San Jose: 3:47pm